<h1>Содержание<span class="tocSkip"></span></h1>
><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия-без-баланса" data-toc-modified-id="Логистическая-регрессия-без-баланса-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Логистическая регрессия без баланса</a></span></li><li><span><a href="#Логистическая-регрессия-с-балансом" data-toc-modified-id="Логистическая-регрессия-с-балансом-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Логистическая регрессия с балансом</a></span></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

Наша задача построить модель для определения токсичных комментариев на основе комментов предложенных заказчиком. При этом метрики качества F1 не меньше 0.75. Работать будем не с BERT.

## Подготовка

In [ ]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeRegressor
# импортируем библиотеки

In [ ]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet') # скачиваем нужные материалы.

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data.head() # Смотрим данные.

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
data['text'].duplicated().sum()
# Смотрим если дубликаты строк

0

In [ ]:
data.info() # Количество записей 159292

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [ ]:
data['toxic'].describe() # 10 процентов токсичных

count    159292.000000
mean          0.101612
std           0.302139
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: toxic, dtype: float64

In [ ]:
data = data.drop('Unnamed: 0', axis=1)

In [ ]:
# Инициализация стоп-слов
stop_words = list(stopwords.words('english'))

In [ ]:
# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление знаков пунктуации
    text = re.sub(r'[^\w\s]', '', text)
    # Удаление цифр
    text = re.sub(r'\d+', '', text)
    # Токенизация
    tokens = nltk.tokenize.word_tokenize(text)
    # Удаление стоп-слов и лемматизация
    tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens if token not in stop_words]
    # Объединение токенов в строку
    text = ' '.join(tokens)
    return text

In [ ]:
# Применение предобработки к колонке "text"
data['lemma_text'] = data['text'].apply(preprocess_text)

In [ ]:
data.head() # Смотрим данные.

,text,toxic,lemma_text
0,Explanation\nWhy the edits made under my usern...,0,explanation edits make username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,daww match background colour im seemingly stuc...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man im really try edit war guy constantly ...
3,"""\nMore\nI can't make any real suggestions on ...",0,cant make real suggestion improvement wonder s...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page thats


In [ ]:
features = data['lemma_text'] # выделяем признаки и целевой признак
target = data['toxic']

In [ ]:
# Разделение на обучающую и тестовую выборки c stratify=target
X_train, X_test, y_train, y_test = train_test_split(
    features,
    target,
    test_size=0.2,
    random_state=12345,
    stratify=target
)

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stop_words)

X_train_features = vectorizer.fit_transform(X_train)

# Преобразование тестовой выборки с использованием тех же признаков
X_test_features = vectorizer.transform(X_test)

In [ ]:
X_train_features.shape # размер матрицы

(127433, 179644)

Данные рассмотрели, преобразовали в нужный вид и теперь они готовы к обучению.

## Обучение

### Случайный лес

In [ ]:
pipe = Pipeline([
    ('tfid',TfidfVectorizer(stop_words=stop_words)),
    ('model_rf', RandomForestClassifier(random_state=12345))
])

In [ ]:
params = {'model_rf__max_depth':range(4,10),
          'model_rf__n_estimators': range(50, 100, 10)
         }
grid_rf = GridSearchCV(pipe, params, cv=3, scoring='f1', n_jobs=-1)
grid_rf.fit(X_train, y_train);
print('Лучшие параметры для случайного леса:', grid_rf.best_params_)
print('Оценка кацества модели f1_score:', grid_rf.best_score_)

Лучшие параметры для случайного леса: {'model_rf__max_depth': 4, 'model_rf__n_estimators': 50}
Оценка кацества модели f1_score: 0.0


Возможно, лес предсказывает для всех входов один и тот же класс, поэтому f1=0. То есть модель ничего хорошего выучить просто не смогла

### Логистическая регрессия

#### Логистическая регрессия без баланса

In [ ]:
pipe = Pipeline([
    ('tfid',TfidfVectorizer(stop_words=stop_words)),
    ('model_lr', LogisticRegression(random_state=12345))
])

In [ ]:
parameters = {'model_lr__C':[0.01, 0.1, 1, 10, 100],
              'model_lr__max_iter':[500]}
grid = GridSearchCV(pipe, parameters, cv=3, scoring='f1', n_jobs=-1)
grid.fit(X_train, y_train);
print('Лучшие параметры для LogisticRegression:', grid.best_params_)
print('Оценка кацества модели f1:', grid.best_score_)

Лучшие параметры для LogisticRegression: {'model_lr__C': 10, 'model_lr__max_iter': 500}
Оценка кацества модели f1: 0.7655589674357776


Лучшие параметры для LogisticRegression: {'model_lr__C': 10, 'model_lr__penalty': 'l2'}  
Оценка кацества модели f1: 0.7655589674357776

#### Логистическая регрессия с балансом

In [ ]:
parameters = {'model_lr__C':[0.01, 0.1, 1, 10, 100],
              'model_lr__class_weight': ['balanced'],
              'model_lr__max_iter':[500]
             }
grid = GridSearchCV(pipe, parameters, cv=3, scoring='f1', n_jobs=-1)
grid.fit(X_train, y_train);
print('Лучшие параметры для LogisticRegression:', grid.best_params_)
print('Оценка кацества модели f1:', grid.best_score_)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: T

Лучшие параметры для LogisticRegression: {'model_lr__C': 10, 'model_lr__class_weight': 'balanced', 'model_lr__penalty': 'l2'}
Оценка кацества модели f1: 0.7599526225566077


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Лучшие параметры для LogisticRegression: {'model_lr__C': 10, 'model_lr__class_weight': 'balanced', 'model_lr__penalty': 'l2'}
Оценка кацества модели f1: 0.7599526225566077

Лучший результат показала модель логистичекая регрессия без баланса при f1= 0.7655285650892871. Не использовал модели град.бустинга, потому что, времени очень много проверяет.

## Выводы

Проверим модель на тестовых данных

In [ ]:
model = LogisticRegression(random_state=12345, C=10, max_iter=500)
model.fit(X_train_features, y_train)
prediction = model.predict(X_test_features)
print('f1_score:', f1_score(y_test, prediction))

f1_score: 0.7771878746360679


На тестовых данных модель показала хороший результат f1_score: 0.7771878746360679.

Мы получили навыки обработки, преобразования текста в признаки для обучения моделей.  
Обучили модели для предсказывания результата токсичности комментария.  
Лучший результат показала Логистическая регрессия без баланса f1_score= 0.7655589674357776, что согласно ТЗ удвлетворяет заказчика.